<a href="https://colab.research.google.com/github/rsudiplive/Method_Of_Max_Likelihood_Logistic_Regression/blob/main/Max_Likelihood_Estimation_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What do we mean by Maximum Likelihood estimation? 

The **maximum likelihood estimation** is a method that determines values for parameters of the model. It is the statistical method of estimating the parameters of the probability distribution by maximizing the likelihood function. The point in which the parameter value that maximizes the likelihood function is called the maximum likelihood estimate. <br>

**Goal:** <br>
Maximum Likelihood Estimation involves treating the problem as an optimization or search problem, where we seek a set of parameters that results in the best fit for the joint probability of the data sample (X). <br>

**Five Major Steps in MLE:**

1. Perform a certain experiment to collect the data.
2. Choose a parametric model of the data, with certain modifiable parameters.
3. Formulate the likelihood as an objective function to be maximized.
4. Maximize the objective function and derive the parameters of the model. 



**Deriving MLE from scratch:**

In [17]:
#importing the libraries
import math
import random

import numpy as np
from numpy import log, exp
import pandas as pd

In [2]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self): #The __str__ method in Python represents the class objects as a string. This method returns the str obj!
        return "{0},{1}".format(self.x, self.y)

In [9]:
points = [(Point(row.x, row.y)) for index, row in pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dat/simple_logistic_regression.csv").iterrows()]

best_likelihood = -10_000_000
b0 = .01
b1 = .01


# calculating maximum likelihood

def predict_probability(x):
    p = 1.0 / (1.0001 + math.exp(-(b0 + b1 * x)))
    return p


for i in range(1_000_000):

    # Select b0 or b1 randomly, and adjust it randomly
    random_b = random.choice(range(2))

    random_adjust = np.random.normal()

    if random_b == 0:
        b0 += random_adjust
    elif random_b == 1:
        b1 += random_adjust

    # Calculate total likelihood
    true_estimates = sum(math.log(predict_probability(p.x)) for p in points if p.y == 1.0)
    false_estimates = sum(math.log(1.0 - predict_probability(p.x)) for p in points if p.y == 0.0)

    total_likelihood = true_estimates + false_estimates

    # If likelihood improves, keep the random adjustment. Otherwise revert.
    if best_likelihood < total_likelihood:
        best_likelihood = total_likelihood
    elif random_b == 0:
        b0 -= random_adjust
    elif random_b == 1:
        b1 -= random_adjust

print("1.0 / (1 + exp(-({0} + {1}*x))".format(b0, b1))
print("BEST LIKELIHOOD: {0}".format(math.exp(best_likelihood)))

1.0 / (1 + exp(-(-3.175516491269391 + 0.6926650530853975*x))
BEST LIKELIHOOD: 4.790311041879937e-05


Now, let's move forward with the python implementation of finding the maximum likelihood with the famous employee retention problem once used by IBM!

In [10]:
class EmployeeRetention:
    def __init__(self, sex, age, promotions, years_employed, did_quit):
        self.sex = sex
        self.age = age
        self.promotions = promotions
        self.years_employed = years_employed
        self.did_quit = did_quit

In [11]:
employee_data = [(EmployeeRetention(row[0], row[1], row[2], row[3], row[4])) for index, row in
                 pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dat/employee_retention_analysis.csv").iterrows()]

In [14]:
best_likelihood = -100_000_000_000.0
b0 = 1.0  # constant
b1 = 1.0  # sex beta
b2 = 1.0  # age beta
b3 = 1.0  # promotions beta
b4 = 1.0  # years employed beta

iterations = 50_000

In [15]:
# calculate maximum likelihood

def predict_probability(sex, age, promotions, years_employed):
    x = b0 + (b1 * sex) + (b2 * age) + (b3 * promotions) + (b4 * years_employed)
    odds = exp(-x)
    p = 1.0 / (1.0 + odds)
    return p

In [18]:
for i in range(iterations):

    # Select b0, b1, b2, b3, or b4 randomly, and adjust it by a random amount
    random_b = random.choice(range(5))

    random_adjust = np.random.standard_normal()

    if random_b == 0:
        b0 += random_adjust
    elif random_b == 1:
        b1 += random_adjust
    elif random_b == 2:
        b2 += random_adjust
    elif random_b == 3:
        b3 += random_adjust
    elif random_b == 4:
        b4 += random_adjust

    # calculate new likelihood
    # Use logarithmic addition to avoid multiplication and decimal underflow
    new_likelihood = 0.0

    for emp in employee_data:

        probability = predict_probability(emp.sex, emp.age, emp.promotions, emp.years_employed)

        if emp.did_quit == 1:
            new_likelihood += log(probability)
        else:
            new_likelihood += log(1.00001 - probability)

    # If solution improves, keep it and make it new best likelihood. Otherwise undo the adjustment
    if best_likelihood < new_likelihood:
        best_likelihood = new_likelihood
    elif random_b == 0:
        b0 -= random_adjust
    elif random_b == 1:
        b1 -= random_adjust
    elif random_b == 2:
        b2 -= random_adjust
    elif random_b == 3:
        b3 -= random_adjust
    elif random_b == 4:
        b4 -= random_adjust

# Print best result
print("1.0 / (1 + exp(-({0} + {1}*s + {2}*a + {3}*p + {4}*y))".format(b0, b1, b2, b3, b4))
print("BEST LIKELIHOOD: {0}".format(math.exp(best_likelihood)))

1.0 / (1 + exp(-(1.3846018003331872 + 0.16471814439041688*s + -0.1320930856224285*a + -2.386172473632742*p + 1.230839164904981*y))
BEST LIKELIHOOD: 9.799640748875159e-13


In [37]:
# Interact and test with new employee data
def predict_employee_will_stay(sex, age, promotions, years_employed):
    probability_of_leaving = predict_probability(sex, age, promotions, years_employed)
    if probability_of_leaving >= .5:
        return "WILL LEAVE, {0}% chance of leaving".format(round(probability_of_leaving * 100.0,2))
    else:
        return "WILL STAY, {0}% chance of leaving".format(round(probability_of_leaving * 100.0,2))

while True:
    n = input("Predict employee will stay or leave {sex},{age},{promotions},{years employed}: ")
    (sex, age, promotions, years_employed) = n.split(",")
    print(predict_employee_will_stay(int(sex), int(age), int(promotions), int(years_employed)))
    break

Predict employee will stay or leave {sex},{age},{promotions},{years employed}: 1,34,2,11
WILL LEAVE, 99.71% chance of leaving


So, we can play around with this and find out whether an employee will leave or stay in an organization by the method of maximum likelihood!